In [1]:
# coding: UTF-8
import time
import torch
import numpy as np
import torch.utils.data as Data
from torch.utils.data import SubsetRandomSampler
import librosa
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft
import warnings
warnings.filterwarnings("ignore")
import librosa.display
from train_eval import train, init_network
from importlib import import_module
import argparse

/home/linjw/anaconda3/envs/scrapy/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 导入训练好的模型

In [2]:
model_name = "TextCNN_custom"
x = import_module(f'models.{model_name}')
config = x.Config()
model = x.Model(config).to(config.device)
model=torch.load('./model_backup/CNN_CCC_0.61/tmodel_0.pkl')

### 抽取wav音频文件

In [8]:
from moviepy.editor import *
video = VideoFileClip('/home/linjw/code2/gnn_study_crawler/Voice_arousal/movie.mp4')
audio = video.audio
audio.write_audiofile('./result/test.wav')
# audio.write_audiofile('test.mp3')

MoviePy - Writing audio in ./result/test.wav


MoviePy - Done.


### 输入数据处理

In [9]:
def get_waveform(x):
    try:
        # np.append(temp,np.array([[x.P]*1600]), axis = 0)
        temp = x.reshape(100,1600)
        mu = np.mean(temp, axis=0)
        sigma = np.std(temp, axis=0)
        return (temp - mu) / sigma
    except:
        return None
def get_spectrogram(x):
    try:
        temp=librosa.amplitude_to_db(librosa.stft(x.flatten())).reshape(1025, 313)
        mu = np.mean(temp, axis=0)
        sigma = np.std(temp, axis=0)
        return (temp - mu) / sigma
    except:
        return None

samples, sr = librosa.load('./result/test.wav', sr=16000)
step = 160000
wave_data = [samples[i:i+step] for i in range(0,len(samples),step)]
wave_data[-1] = samples[-step:]
wave_data = np.array([get_waveform(x) for x in wave_data])
spect_data = np.array([get_spectrogram(x) for x in wave_data])
print(wave_data.shape,spect_data.shape)

(11, 100, 1600) (11, 1025, 313)


In [10]:
class MyDataSet(Data.Dataset):

    def __init__(self):
        self.x = wave_data,spect_data

    def __getitem__(self, index):
        return self.x[0][index],self.x[1][index],0

    def __len__(self):
        return len(self.x[0])
data=MyDataSet()

In [11]:
dev_iter = Data.DataLoader(data, batch_size=1000, shuffle=False)

In [12]:
## 模型预测
model.eval()
alist=[]
for trains1,trains2, labels in dev_iter:
    trains1, trains12, labels = trains1.cuda().float(),trains2.cuda().float(), labels.float().cuda()
    outputs = model(trains1,trains12)  # 10,73->10,5
    alist=list(outputs.flatten().cpu().detach().numpy())
    olist=[np.expm1(x) for x in list(outputs.flatten().cpu().detach().numpy())]
sum(olist)/len(olist)

-0.4143091831356287

In [8]:
# coding: UTF-8
import torch
import numpy as np
import torch.utils.data as Data
import librosa
import numpy as np
from scipy.fft import fft
import warnings
import pandas as pd
warnings.filterwarnings("ignore")
import librosa.display
from moviepy.editor import *
from importlib import import_module
model_name = "TextCNN_custom"


x = import_module(f'models.{model_name}')
config = x.Config()
device = torch.device("cpu")
model = x.Model(config).to(device)
model  =torch.load('./ArousalRegress/model_backup/CNN_CCC_0.61/tmodel_0.pkl').to(device)

def get_waveform(x):
    try:
        # np.append(temp,np.array([[x.P]*1600]), axis = 0)
        temp = x.reshape(100,1600)
        mu = np.mean(temp, axis=0)
        sigma = np.std(temp, axis=0)
        return (temp - mu) / sigma
    except:
        return None
def get_spectrogram(x):
    try:
        temp=librosa.amplitude_to_db(librosa.stft(x.flatten())).reshape(1025, 313)
        mu = np.mean(temp, axis=0)
        sigma = np.std(temp, axis=0)
        return (temp - mu) / sigma
    except:
        return None

class MyDataSet(Data.Dataset):

    def __init__(self,wave_data,spect_data):
        self.x = wave_data,spect_data

    def __getitem__(self, index):
        return self.x[0][index],self.x[1][index],0

    def __len__(self):
        return len(self.x[0])


FileNotFoundError: [Errno 2] No such file or directory: './ArousalRegress/model_backup/CNN_CCC_0.61/tmodel_0.pkl'

In [ ]:
rawdata=pd.read_excel('/home/linjw/code2/gnn_study_crawler/backup/variation_record.xlsx',dtype=str)
rawdata=rawdata.drop_duplicates(subset='aweme_id').head(5)
def get_arousal(id):
    video = VideoFileClip(f'./video/{id}.mp4')
    audio = video.audio
    audio.write_audiofile(f'./voice/test{id}.wav')
    samples, sr = librosa.load(f'./voice/test{id}.wav', sr=16000)
    step = 160000
    wave_data = [samples[i:i+step] for i in range(0,len(samples),step)]
    wave_data[-1] = samples[-step:]
    wave_data = np.array([get_waveform(x) for x in wave_data])
    spect_data = np.array([get_spectrogram(x) for x in wave_data])
    print(wave_data.shape,spect_data.shape)

    data=MyDataSet(wave_data,spect_data)
    dev_iter = Data.DataLoader(data, batch_size=1000, shuffle=False)
    ## 模型预测
    model.eval()
    alist=[]
    for trains1,trains2, labels in dev_iter:
        trains1, trains12, labels = trains1.float(),trains2.float(), labels.float()
        outputs = model(trains1,trains12)  # 10,73->10,5
        alist=list(outputs.flatten().detach().numpy())
        olist=[np.expm1(x) for x in list(outputs.flatten().detach().numpy())]
    print(sum(olist)/len(olist))
rawdata.rawdata.apply(get_arousal)